### Implementação da Camada Gold

Este notebook representa a etapa final do pipeline de Engenharia de Dados, onde os dados técnicos tratados na camada Silver são transformados em Informação de Negócio. O objetivo desta camada é facilitar o consumo por analistas e ferramentas de visualização, pré-calculando métricas e aplicando regras de negócio específicas.

A partir da tabela limpa da Silver, foi criada a tabela analítica stroke_analytics_gold aplicando-se a Modelagem Dimensional para transformar variáveis contínuas em categorias de negócio, facilitando a análise visual. Exemplos incluem:

**Agrupamento Etário:** Conversão da idade numérica em faixas (Criança, Jovem, Meia Idade, Idoso).

**Classificação Clínica:** Segmentação de pacientes baseada em padrões médicos para Glicose (Diabetes, Pré-diabetes) e IMC (Obesidade, Sobrepeso).

**Perfil de Risco Combinado:** Criação de uma variável sintética que identifica pacientes de "Alto Risco" (possuem hipertensão e/ou doença cardíaca), permitindo análises de correlação multivariada.

In [0]:
%sql
-- 1. Criação da Tabela Gold (Analítica Completa)
CREATE OR REPLACE TABLE mvpengenharia.gold.stroke_analytics_gold
USING DELTA
COMMENT 'Camada Gold: Tabela analítica consolidada. Contém métricas numéricas para correlação e dimensões categóricas para perfis de risco.'
AS
SELECT
  -- DADOS ORIGINAIS (Para Correlações e Estatísticas) ---
  gender,
  age,                
  hypertension,       
  heart_disease,      
  work_type,          
  Residence_type,     
  avg_glucose_level,  
  bmi,                
  smoking_status,     
  stroke,             
  
  -- ENRIQUECIMENTO
  
  -- Faixas Etárias
  CASE 
    WHEN age < 18 THEN '0-17 (Criança/Adolescente)'
    WHEN age BETWEEN 18 AND 39 THEN '18-39 (Jovem Adulto)'
    WHEN age BETWEEN 40 AND 59 THEN '40-59 (Meia Idade)'
    WHEN age >= 60 THEN '60+ (Idoso)'
    ELSE 'Desconhecido'
  END as faixa_etaria,

  -- Classificação de Glicose
  CASE 
    WHEN avg_glucose_level <= 99 THEN 'Normal'
    WHEN avg_glucose_level BETWEEN 100 AND 125 THEN 'Pré-Diabetes'
    WHEN avg_glucose_level > 125 THEN 'Diabetes'
    ELSE 'Desconhecido' 
  END as categoria_glicose,

  -- Classificação de IMC
  CASE 
    WHEN bmi < 18.5 THEN 'Abaixo do Peso'
    WHEN bmi BETWEEN 18.5 AND 24.9 THEN 'Peso Normal'
    WHEN bmi BETWEEN 25 AND 29.9 THEN 'Sobrepeso'
    WHEN bmi >= 30 THEN 'Obesidade'
    ELSE 'Desconhecido'
  END as categoria_imc,

  -- Perfil Combinado (Risco Cardíaco)
  CASE 
    WHEN hypertension = 1 AND heart_disease = 1 THEN 'Altíssimo Risco (Ambos)'
    WHEN hypertension = 1 OR heart_disease = 1 THEN 'Alto Risco (Um dos dois)'
    ELSE 'Baixo Risco (Nenhum)'
  END as perfil_clinico,
  
  -- Auxiliar para contagem
  1 as contador_pacientes,
  current_timestamp() as gold_ingestion_date

FROM mvpengenharia.silver.stroke_dataset_silver;

num_affected_rows,num_inserted_rows


Abaixo é executada a documentação do Catálogo de Dados. Diferente das camadas anteriores, as descrições aqui focam na Regra de Negócio (ex: explicar que a categoria "Diabetes" considera glicose > 125), garantindo que os consumidores dos dados compreendam o significado analítico de cada coluna sem precisar ler o código.

In [0]:
%sql
-- 2. Documentação e Descrições (Data Catalog)
ALTER TABLE mvpengenharia.gold.stroke_analytics_gold ALTER COLUMN gender COMMENT 'Gênero do paciente';
ALTER TABLE mvpengenharia.gold.stroke_analytics_gold ALTER COLUMN age COMMENT 'Idade numérica';
ALTER TABLE mvpengenharia.gold.stroke_analytics_gold ALTER COLUMN work_type COMMENT 'Tipo de vínculo trabalhista';
ALTER TABLE mvpengenharia.gold.stroke_analytics_gold ALTER COLUMN Residence_type COMMENT 'Local de residência (Rural/Urbano)';
ALTER TABLE mvpengenharia.gold.stroke_analytics_gold ALTER COLUMN avg_glucose_level COMMENT 'Nível médio de glicose (Numérico)';
ALTER TABLE mvpengenharia.gold.stroke_analytics_gold ALTER COLUMN bmi COMMENT 'Índice de Massa Corporal (Númerico, Nulos preenchidos com a Mediana)';
ALTER TABLE mvpengenharia.gold.stroke_analytics_gold ALTER COLUMN smoking_status COMMENT 'Status de tabagismo (Nulos preenchidos como Unknown)';
ALTER TABLE mvpengenharia.gold.stroke_analytics_gold ALTER COLUMN stroke COMMENT 'Target: 1 = AVC, 0 = Não AVC';
ALTER TABLE mvpengenharia.gold.stroke_analytics_gold ALTER COLUMN hypertension COMMENT 'Indicador binário de hipertensão (0=Não, 1=Sim)';
ALTER TABLE mvpengenharia.gold.stroke_analytics_gold ALTER COLUMN heart_disease COMMENT 'Indicador binário de doença cardíaca (0=Não, 1=Sim)';

-- 2.1 Descrições das Colunas Calculadas (Business Rules)
ALTER TABLE mvpengenharia.gold.stroke_analytics_gold ALTER COLUMN faixa_etaria COMMENT 'Agrupamento: 0-17, 18-39, 40-59, 60+';
ALTER TABLE mvpengenharia.gold.stroke_analytics_gold ALTER COLUMN categoria_glicose COMMENT 'Classificação: Normal, Pré-Diabetes, Diabetes (>125)';
ALTER TABLE mvpengenharia.gold.stroke_analytics_gold ALTER COLUMN categoria_imc COMMENT 'Classificação: Abaixo, Normal, Sobrepeso, Obesidade (>=30)';
ALTER TABLE mvpengenharia.gold.stroke_analytics_gold ALTER COLUMN perfil_clinico COMMENT 'Risco Combinado: Verifica se possui Hipertensão OU Doença Cardíaca';

-- 2.2 Metadados
ALTER TABLE mvpengenharia.gold.stroke_analytics_gold ALTER COLUMN contador_pacientes COMMENT 'Auxiliar para contagens (Valor fixo 1)';
ALTER TABLE mvpengenharia.gold.stroke_analytics_gold ALTER COLUMN gold_ingestion_date COMMENT 'Data de processamento da camada Gold';

Serão executadas consultas de validação agrupando os dados pelas novas dimensões criadas. Isso servirá para confirmar a consistência das regras de negócio (ex: verificar se os grupos de "Alto Risco" realmente apresentam taxas maiores de AVC) e assegurar que a tabela final está pronta para responder às perguntas de negócio definidas no objetivo do projeto.

Validação dos Fatores Fisiológicos (Idade, Glicose e IMC)
Adicionada a categoria_imc na query original. Isso mostra, por exemplo, se existem muitos "Idosos" com "Obesidade" e "Diabetes" que tiveram AVC.

In [0]:
%sql
SELECT 
  faixa_etaria,
  categoria_glicose,
  categoria_imc, -- Nova classificação que criada
  count(*) as total_pacientes,
  sum(stroke) as total_avc,
  -- Calcula a taxa % de AVC nesse grupo
  round((sum(stroke) / count(*)) * 100, 2) as taxa_avc_percentual
FROM mvpengenharia.gold.stroke_analytics_gold
GROUP BY 1, 2, 3
ORDER BY 1, 2, 3;

faixa_etaria,categoria_glicose,categoria_imc,total_pacientes,total_avc,taxa_avc_percentual
0-17 (Criança/Adolescente),Desconhecido,Abaixo do Peso,43,0,0.0
0-17 (Criança/Adolescente),Desconhecido,Obesidade,13,0,0.0
0-17 (Criança/Adolescente),Desconhecido,Peso Normal,56,0,0.0
0-17 (Criança/Adolescente),Desconhecido,Sobrepeso,11,0,0.0
0-17 (Criança/Adolescente),Diabetes,Abaixo do Peso,243,0,0.0
0-17 (Criança/Adolescente),Diabetes,Obesidade,46,0,0.0
0-17 (Criança/Adolescente),Diabetes,Peso Normal,294,0,0.0
0-17 (Criança/Adolescente),Diabetes,Sobrepeso,83,0,0.0
0-17 (Criança/Adolescente),Normal,Abaixo do Peso,1935,0,0.0
0-17 (Criança/Adolescente),Normal,Obesidade,371,1,0.27


Validação dos Fatores de Comportamento e Histórico
Esta é uma consulta fundamental para validar as colunas adicionadas por último (smoking_status e perfil_clinico). Ela responde diretamente as perguntas sobre tabagismo e comorbidades.

In [0]:
%sql
SELECT 
  perfil_clinico, -- Alto Risco vs Baixo Risco
  smoking_status, -- Incluindo os Unknowns que tratamos
  count(*) as total_pacientes,
  sum(stroke) as total_avc,
  round((sum(stroke) / count(*)) * 100, 2) as taxa_avc_percentual
FROM mvpengenharia.gold.stroke_analytics_gold
GROUP BY 1, 2
ORDER BY taxa_avc_percentual DESC; -- Ordena do maior risco para o menor

perfil_clinico,smoking_status,total_pacientes,total_avc,taxa_avc_percentual
Altíssimo Risco (Ambos),Unknown,63,9,14.29
Altíssimo Risco (Ambos),never smoked,174,18,10.34
Altíssimo Risco (Ambos),smokes,120,11,9.17
Altíssimo Risco (Ambos),formerly smoked,158,14,8.86
Alto Risco (Um dos dois),formerly smoked,1375,86,6.25
Alto Risco (Um dos dois),smokes,923,48,5.2
Alto Risco (Um dos dois),Unknown,791,40,5.06
Alto Risco (Um dos dois),never smoked,2004,99,4.94
Baixo Risco (Nenhum),formerly smoked,5960,121,2.03
Baixo Risco (Nenhum),smokes,5519,74,1.34


Verificação da tabela final abaixo.

In [0]:
%sql
SELECT * FROM mvpengenharia.gold.stroke_analytics_gold 
LIMIT 15;

gender,age,hypertension,heart_disease,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke,faixa_etaria,categoria_glicose,categoria_imc,perfil_clinico,contador_pacientes,gold_ingestion_date
Male,3,0,0,children,Rural,95.12,18.0,Unknown,0,0-17 (Criança/Adolescente),Normal,Abaixo do Peso,Baixo Risco (Nenhum),1,2025-12-13T20:44:08.924Z
Male,58,1,0,Private,Urban,87.96,39.2,never smoked,0,40-59 (Meia Idade),Normal,Obesidade,Alto Risco (Um dos dois),1,2025-12-13T20:44:08.924Z
Female,8,0,0,Private,Urban,110.89,17.6,Unknown,0,0-17 (Criança/Adolescente),Pré-Diabetes,Abaixo do Peso,Baixo Risco (Nenhum),1,2025-12-13T20:44:08.924Z
Female,70,0,0,Private,Rural,69.04,35.9,formerly smoked,0,60+ (Idoso),Normal,Obesidade,Baixo Risco (Nenhum),1,2025-12-13T20:44:08.924Z
Male,14,0,0,Never_worked,Rural,161.28,19.1,Unknown,0,0-17 (Criança/Adolescente),Diabetes,Peso Normal,Baixo Risco (Nenhum),1,2025-12-13T20:44:08.924Z
Female,47,0,0,Private,Urban,210.95,50.1,Unknown,0,40-59 (Meia Idade),Diabetes,Obesidade,Baixo Risco (Nenhum),1,2025-12-13T20:44:08.924Z
Female,52,0,0,Private,Urban,77.59,17.7,formerly smoked,0,40-59 (Meia Idade),Normal,Abaixo do Peso,Baixo Risco (Nenhum),1,2025-12-13T20:44:08.924Z
Female,75,0,1,Self-employed,Rural,243.53,27.0,never smoked,0,60+ (Idoso),Diabetes,Sobrepeso,Alto Risco (Um dos dois),1,2025-12-13T20:44:08.924Z
Female,32,0,0,Private,Rural,77.67,32.3,smokes,0,18-39 (Jovem Adulto),Normal,Obesidade,Baixo Risco (Nenhum),1,2025-12-13T20:44:08.924Z
Female,74,1,0,Self-employed,Urban,205.84,54.6,never smoked,0,60+ (Idoso),Diabetes,Obesidade,Alto Risco (Um dos dois),1,2025-12-13T20:44:08.924Z
